In [1]:
import requests
import pandas as pd

Opendota_api_heros_url = "https://api.opendota.com/api/heroes"
print(Opendota_api_heros_url)
r = requests.get(Opendota_api_heros_url, timeout=3)
r_json = r.json()
en_hero_df = pd.DataFrame(r_json)
en_hero_df.head(3)

https://api.opendota.com/api/heroes


,id,name,localized_name,primary_attr,attack_type,roles,legs
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
1,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler, Carry]",2
2,3,npc_dota_hero_bane,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",4


In [2]:
en_hero_df=en_hero_df.rename(columns={"localized_name": "en_official_name", "id": "hero_id", "name": "ingame_name"})

Steam_key = '15CC480338F0DD1DB2E875618D8072F2'
Valve_api_get_heros_url = f"https://api.steampowered.com/IEconDOTA2_570/GetHeroes/v0001/?key={Steam_key}&language=zh-cn"

r = requests.get(Valve_api_get_heros_url, timeout=3)
r_json = r.json()
cn_hero_df = pd.DataFrame(r_json["result"]["heroes"])

hero_df = pd.merge(cn_hero_df.rename(columns = {'name':'ingame_name','localized_name':'hero_name'}),en_hero_df,on='ingame_name')
hero_df.head(3)

,ingame_name,id,hero_name,hero_id,en_official_name,primary_attr,attack_type,roles,legs
0,npc_dota_hero_antimage,1,敌法师,1,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
1,npc_dota_hero_axe,2,斧王,2,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler, Carry]",2
2,npc_dota_hero_bane,3,祸乱之源,3,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",4


In [3]:

df2 = hero_df.loc[:,['hero_id', 'ingame_name', 'en_official_name', 'primary_attr',
               'attack_type', 'legs', 'hero_name']]
df2.head(3)

,hero_id,ingame_name,en_official_name,primary_attr,attack_type,legs,hero_name
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,2,敌法师
1,2,npc_dota_hero_axe,Axe,str,Melee,2,斧王
2,3,npc_dota_hero_bane,Bane,int,Ranged,4,祸乱之源


前面是拉去了所有英雄信息，后面会拉去比赛记录把每场比赛的阵容给展示出来

In [4]:
Opendota_heros_sql = "SELECT matches.match_id, picks_bans FROM matches WHERE matches.match_id = '6227492909' AND matches.start_time >= extract(epoch from timestamp '2021-09-30T16:00:00.000Z')"
Opendota_heros_sqlquery_url = "https://api.opendota.com/api/explorer?sql=" + Opendota_heros_sql
print(Opendota_heros_sqlquery_url)
r = requests.get(Opendota_heros_sqlquery_url, timeout=3)
r_json = r.json()

https://api.opendota.com/api/explorer?sql=SELECT matches.match_id, picks_bans FROM matches WHERE matches.match_id = '6227492909' AND matches.start_time >= extract(epoch from timestamp '2021-09-30T16:00:00.000Z')


In [5]:

r_json["rows"][0]["picks_bans"]
r_json_new = [x for x in r_json["rows"][0]["picks_bans"] if x["is_pick"] == True]
r_json_new

[{'is_pick': True, 'hero_id': 19, 'team': 1, 'order': 4},
 {'is_pick': True, 'hero_id': 97, 'team': 0, 'order': 5},
 {'is_pick': True, 'hero_id': 3, 'team': 0, 'order': 6},
 {'is_pick': True, 'hero_id': 77, 'team': 1, 'order': 7},
 {'is_pick': True, 'hero_id': 112, 'team': 0, 'order': 14},
 {'is_pick': True, 'hero_id': 23, 'team': 1, 'order': 15},
 {'is_pick': True, 'hero_id': 101, 'team': 1, 'order': 16},
 {'is_pick': True, 'hero_id': 106, 'team': 0, 'order': 17},
 {'is_pick': True, 'hero_id': 58, 'team': 1, 'order': 22},
 {'is_pick': True, 'hero_id': 109, 'team': 0, 'order': 23}]

In [8]:
radiant_lanes = [ bp for bp in r_json_new if 1 == bp['team']]
radiant_lanes

[{'is_pick': True, 'hero_id': 19, 'team': 1, 'order': 4},
 {'is_pick': True, 'hero_id': 77, 'team': 1, 'order': 7},
 {'is_pick': True, 'hero_id': 23, 'team': 1, 'order': 15},
 {'is_pick': True, 'hero_id': 101, 'team': 1, 'order': 16},
 {'is_pick': True, 'hero_id': 58, 'team': 1, 'order': 22}]

In [11]:
dire_lanes = [ bp for bp in r_json_new if bp['team'] == 0 ]
dire_lanes

[{'is_pick': True, 'hero_id': 97, 'team': 0, 'order': 5},
 {'is_pick': True, 'hero_id': 3, 'team': 0, 'order': 6},
 {'is_pick': True, 'hero_id': 112, 'team': 0, 'order': 14},
 {'is_pick': True, 'hero_id': 106, 'team': 0, 'order': 17},
 {'is_pick': True, 'hero_id': 109, 'team': 0, 'order': 23}]